In [ ]:
import sys
import json
import time
import pandas as pd
import matplotlib.pyplot as plt
import numpy as np
from pathlib import Path

# Add the app directory to the path
sys.path.append('app')

from app.models import Course, Teacher, Classroom, StudentGroup, Timeslot, Constraint
from app.services.GeneticScheduler import GeneticScheduler


In [ ]:
# Generate test data if it doesn't exist
import subprocess
import os

if not os.path.exists('test_data.json'):
    print("Generating test data...")
    subprocess.run([sys.executable, 'test_data_extractor.py'])
else:
    print("Using existing test data...")


In [ ]:
# Load test data
with open('test_data.json', 'r') as f:
    test_data = json.load(f)

print("Test data loaded:")
for key, value in test_data.items():
    print(f"  - {key}: {len(value)} items")


In [ ]:
# Convert test data to model objects
def create_model_objects(data):
    """Convert JSON data to model objects"""
    
    # Convert timeslots
    timeslots = [
        Timeslot(code=ts['code'], startTime=ts['startTime'], endTime=ts['endTime'])
        for ts in data['timeslots']
    ]
    
    # Convert classrooms
    classrooms = [
        Classroom(
            classroomId=room['classroomId'],
            name=room['name'],
            capacity=room['capacity'],
            type=room['type']
        )
        for room in data['rooms']
    ]
    
    # Convert teachers
    teachers = [
        Teacher(
            teacherId=teacher['teacherId'],
            name=teacher['name'],
            email=teacher['email']
        )
        for teacher in data['teachers']
    ]
    
    # Convert student groups
    student_groups = [
        StudentGroup(
            studentGroupId=sg['studentGroupId'],
            name=sg['name'],
            size=sg['size']
        )
        for sg in data['studentGroups']
    ]
    
    # Convert courses
    courses = [
        Course(
            courseId=course['courseId'],
            name=course['name'],
            teacherId=course['teacherId'],
            sessionType=course['sessionType'],
            studentGroupIds=course['studentGroupIds']
        )
        for course in data['courses']
    ]
    
    # Convert constraints
    constraints = [
        Constraint(
            constraintId=constraint['constraintId'],
            type=constraint['type'],
            name=constraint['name'],
            description=constraint['description'],
            isHard=constraint['isHard'],
            penalty=constraint['penalty'],
            parameters=constraint['parameters']
        )
        for constraint in data['constraints']
    ]
    
    return timeslots, classrooms, teachers, student_groups, courses, constraints

timeslots, classrooms, teachers, student_groups, courses, constraints = create_model_objects(test_data)
days = ["Monday", "Tuesday", "Wednesday", "Thursday", "Friday"]

print(f"Model objects created:")
print(f"  - Timeslots: {len(timeslots)}")
print(f"  - Classrooms: {len(classrooms)}")
print(f"  - Teachers: {len(teachers)}")
print(f"  - Student Groups: {len(student_groups)}")
print(f"  - Courses: {len(courses)}")
print(f"  - Constraints: {len(constraints)}")


In [ ]:
def run_simple_comparison():
    """Run a simple comparison between old and new approaches"""
    
    print("=== ORIGINAL GA (No Adaptive Features) ===")
    
    # Create original-style scheduler (disable adaptive features manually)
    original_scheduler = GeneticScheduler(
        courses=courses,
        teachers=teachers,
        rooms=classrooms,
        student_groups=student_groups,
        timeslots=timeslots,
        days=days,
        constraints=constraints,
        population_size=30,  # Smaller for faster testing
        gene_mutation_rate=0.1,
        chromosome_mutation_rate=0.2
    )
    
    # Disable adaptive features for baseline
    original_scheduler.heuristic_mutation_probability = 0.0  # Pure random mutation
    original_scheduler.stagnation_counter = 0
    
    print("Running original approach...")
    start_time = time.time()
    orig_solution, orig_fitness, orig_report = original_scheduler.run(generations=500)
    orig_duration = time.time() - start_time
    
    print(f"Original Results:")
    print(f"  - Final Fitness: {orig_fitness:.2f}")
    print(f"  - Hard Violations: {orig_report.total_hard_violations if orig_report else 'N/A'}")
    print(f"  - Soft Penalty: {orig_report.total_soft_penalty if orig_report else 'N/A'}")
    print(f"  - Duration: {orig_duration:.2f}s")
    print(f"  - Stagnation Count: {original_scheduler.stagnation_counter}")
    
    print("\\n=== ENHANCED GA (With Adaptive Features) ===")
    
    # Create enhanced scheduler with all features enabled
    enhanced_scheduler = GeneticScheduler(
        courses=courses,
        teachers=teachers,
        rooms=classrooms,
        student_groups=student_groups,
        timeslots=timeslots,
        days=days,
        constraints=constraints,
        population_size=30,  # Same size for fair comparison
        gene_mutation_rate=0.1,
        chromosome_mutation_rate=0.2
    )
    
    print("Running enhanced approach...")
    start_time = time.time()
    enh_solution, enh_fitness, enh_report = enhanced_scheduler.run(generations=500)
    enh_duration = time.time() - start_time
    
    print(f"Enhanced Results:")
    print(f"  - Final Fitness: {enh_fitness:.2f}")
    print(f"  - Hard Violations: {enh_report.total_hard_violations if enh_report else 'N/A'}")
    print(f"  - Soft Penalty: {enh_report.total_soft_penalty if enh_report else 'N/A'}")
    print(f"  - Duration: {enh_duration:.2f}s")
    print(f"  - Stagnation Count: {enhanced_scheduler.stagnation_counter}")
    print(f"  - Final Heuristic Probability: {enhanced_scheduler.heuristic_mutation_probability:.3f}")
    
    # Calculate improvements
    fitness_improvement = ((orig_fitness - enh_fitness) / orig_fitness * 100) if orig_fitness > 0 else 0
    time_improvement = ((orig_duration - enh_duration) / orig_duration * 100) if orig_duration > 0 else 0
    
    print("\\n=== COMPARISON SUMMARY ===")
    print(f"Fitness Improvement: {fitness_improvement:.1f}% (lower is better)")
    print(f"Time Improvement: {time_improvement:.1f}% (negative means took longer)")
    
    return {
        'original': {'fitness': orig_fitness, 'duration': orig_duration, 'report': orig_report},
        'enhanced': {'fitness': enh_fitness, 'duration': enh_duration, 'report': enh_report},
        'improvements': {'fitness': fitness_improvement, 'time': time_improvement}
    }

# Run the comparison
results = run_simple_comparison()
